In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen= True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    base_model_updated: Path
    params_IMAGE_SIZE: list
    params_CLASSES: int
    params_WEIGHTS: str
    params_LEARNING_RATE: float
    params_INCLUDE_TOP: bool

In [7]:
from src.Kidney_Disease_Classification.constants import *
from src.Kidney_Disease_Classification.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directory([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            base_model_updated = config.base_model_updated,
            params_IMAGE_SIZE = self.params.IMAGE_SIZE,
            params_CLASSES = self.params.CLASSES,
            params_WEIGHTS = self.params.WEIGHTS,
            params_LEARNING_RATE = self.params.LEARNING_RATE,
            params_INCLUDE_TOP = self.params.INCLUDE_TOP

        )

        return prepare_base_model_config

In [9]:
import tensorflow as tf

In [10]:
class PrepareBaseModel:
    def __init__(self, config : PrepareBaseModelConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
            input_shape =self.config.params_IMAGE_SIZE,
            weights=self.config.params_WEIGHTS,
            include_top=self.config.params_INCLUDE_TOP
            )
        self.save_model(path = self.config.base_model_path, model = self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) or (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units= classes,
            activation= "softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_CLASSES,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_LEARNING_RATE
        )
        self.save_model(path = self.config.base_model_updated, model = self.full_model)

    @staticmethod
    def save_model(path: Path, model : tf.keras.Model):
        model.save(path)


In [11]:
try:
   config =  ConfigurationManager()
   prepare_base_model_config = config.get_prepare_base_model_config()
   prepare_base_model = PrepareBaseModel(config= prepare_base_model_config)
   prepare_base_model.get_base_model()
   prepare_base_model.update_base_model()
except Exception as e:
   raise e

[2023-11-15 14:11:44,579  - INFO - common - Reading done from config\config.yaml file]
[2023-11-15 14:11:44,581  - INFO - common - Reading done from params.yaml file]
[2023-11-15 14:11:44,584  - INFO - common - Directory is created for artifacts]
[2023-11-15 14:11:44,584  - INFO - common - Directory is created for artifacts/prepare_base_model]
[2023-11-15 14:11:45,091  - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 22